In [ ]:
!pip install langchain
#pdf내용을 정리해주는 라이브러리
!pip install PyPDF2
!pip install sentence-transformers
!pip install gradio
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 660.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!pip install faiss-cpu
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.0 MB/s eta 0:00:00


In [ ]:
import os

from PyPDF2 import PdfReader

import gradio as gr

#텍스트 전처리 관련 라이브러리
#문서를 "청크화"하는데 쓰이는 라이브러리
from langchain.text_splitter import CharacterTextSplitter
#청크화된 문서를 '임베딩' 하는 데 쓰이는 라이브러리
from langchain.embeddings import HuggingFaceEmbeddings
#벡터스토어, 벡터 db와 관련된 라이브러리
from langchain import FAISS

#LLM과의 통신
#질의 응답 체인
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
#openai에서 보내는 콜백을 다루는 라이브러리
from langchain.callbacks import get_openai_callback

In [ ]:
#텍스트 전처리 수행하는 함수

def preprocess_text(text):

  #characterTextSplitter를 써서 청크로 분할
  text_splitter = CharacterTextSplitter(separator='\n',
                                        chunk_size=1000,
                                        chunk_overlap=200,
                                        length_function=len)
  #위에 정의한 text_splitter로 text를 청크로 만드는 실제 작업
  chunk = text_splitter.split_text(text)

  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  documents = FAISS.from_texts(chunk, embeddings)

  return documents

In [ ]:
def summary(pdf):
  os.environ['OPENAI_API_KEY'] = 's'

  pdf_reader = PdfReader(pdf)
  text = ""

  #for 페이지 in pdf_reader가 읽어오는 콘텐츠
  for page in pdf_reader.pages:
    text += page.extract_text()

    #LLM 사용을 위한 임베딩-벡터화를 거친 text를 documents의 형태로 받다.
    documents = preprocess_text(text)
    query = "업로드된 PDF 파일의 내용을 3~5문장으로 요약하여 주세요. 영어 pdf여도 한글로 요약해주세요."

    if query:
      docs = documents.similarity_search(query)

      llm = ChatOpenAI(model='gpt-3.5-turbo-0301', temperature=0)
      chain = load_qa_chain(llm, chain_type='stuff')

      with get_openai_callback() as cost:
        response = chain.run(input_documents=docs, question=query)

        return response

In [ ]:
import os
from pathlib import Path

In [ ]:
def upload_file(filepath):
  name = Path(filepath).name
  return gr.UploadButton(visible=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
result = summary('/content/drive/MyDrive/천안시청 교육과정/소스코드/[K-DATA] 빅데이터분석기사 실기 체험환경 가이드.pdf')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
result

"I'm sorry, but I cannot summarize the content of a PDF file without knowing what the file is about. Please provide me with the PDF file or at least some context about its content."

In [ ]:
with gr.Blocks() as demo:
    upload_file = gr.File(label="Upload PDF")
    output = gr.Textbox(label="Output")

    #이 사이에 summary() 함수 전체 코드 들어가야 합니다.

    upload_file.change(summary, inputs=upload_file, outputs=output)

# 인터페이스 실행
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://39351c5461d37980d5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
with gr.Blocks() as demo:
    upload_file = gr.File(label="Upload PDF")
    output = gr.Textbox(label="Output")

    def summary(pdf):
      os.environ['OPENAI_API_KEY'] = 'sk-1b-tLAC7xPXlcDmanQi-lgNsgMqHgj7HMg9WmZWZ5KT3BlbkFJvclbK-_UIl7h351UEsZT4KJH8W8ZIf3ZVNRcqgoK4A'

      pdf_reader = PdfReader(pdf)
      text = ""

      #for 페이지 in pdf_reader가 읽어오는 콘텐츠
      for page in pdf_reader.pages:
        text += page.extract_text()

        #LLM 사용을 위한 임베딩-벡터화를 거친 text를 documents의 형태로 받다.
        documents = preprocess_text(text)
        query = "업로드된 PDF 파일의 내용을 3~5문장으로 요약하여 주세요. 영어 pdf여도 한글로 요약해주세요."

        if query:
          docs = documents.similarity_search(query)

          llm = ChatOpenAI(model='gpt-4o', temperature=0)
          chain = load_qa_chain(llm, chain_type='stuff')

          with get_openai_callback() as cost:
            response = chain.run(input_documents=docs, question=query)
            return response

    upload_file.change(summary, inputs=upload_file, outputs=output)

# 인터페이스 실행
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bc7d2e00e2f5d1d8e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
